Define standard ship class

In [102]:
class Ship:
    
    def __init__(self, name, shield, hull, laser, ship_type = 'standard ship'):
        self.name = name
        self.shield = shield
        self.hull = hull
        self.laser = laser
        self.alive = True
        self.ship_type = ship_type
    
    def getName(self):
        return self.name
    
    def getShipType(self):
        return self.ship_type
    
    def getShield(self):
        return self.shield
    
    def getHull(self):
        return self.hull
    
    def getLaser(self):
        return self.laser
    
    def isAlive(self):
        return self.alive
            
    def shoot(self, target, blast = None):
        # set blast initially to None
        # prevent shooting oneself or shooting at dead target
        if (target is not self) and (target.isAlive()):
            if blast is None: # normal kind of laser
                # automatically assume the blast (the amount of energy shot) is equal to the laser strength
                blast = self.getLaser()
            target.shotAt(blast)
        else:
            raise Exception('cannot shoot at this target')
        
    def shotAt(self, blast):
        if self.getShield()>0:
            self.damageShield(blast)
        else:
            self.damageHull(blast)
            
    def damageShield(self, damage):
        if self.getShield() - damage > 0:
            self.shield = self.getShield() - damage
        else:
            left_over = damage - self.getShield()
            self.shield = 0
            self.damageHull(left_over)
            
    def damageHull(self, damage):
        self.hull = self.getHull() - damage/2
        if self.hull <= 0:
            self.alive = False
            
    def __str__(self):
        if self.isAlive():
            return "Name = {}, Type = {}, Hull = {}, Shield = {}, Laser = {}".format(
                self.getName(), self.getShipType(),self.getHull(), self.getShield(), self.getLaser())
        else:
            return "{} is dead".format(self.getName())

Define war ship class

In [103]:
import random
class WarShip(Ship): # inherit from Ship
    
    def __init__(self, name, shield, hull, laser):
        # change the normal constructor by the ship type only
        Ship.__init__(self, name, shield, hull, laser, ship_type = 'war ship')
        
    def shoot(self, target):
        # special attack by missile happens 30% of the time
        num = random.random() # generate a random number between 0 and 1
        if num < 0.3:
            blast = 2*self.getLaser() #special missile has 2 times strength of laser
            Ship.shoot(self, target, blast)
        else:
            # if no special missile attack, then normal shooting
            Ship.shoot(self, target)

Define speeder class

In [104]:
class Speeder(Ship): # inherit from Ship
    
    def __init__(self, name, shield, hull, laser):
        # change the normal constructor by the ship type only
        Ship.__init__(self, name, shield, hull, laser, ship_type = 'speeder')
    
    def shotAt(self,blast):
        num = random.random()
        if num > 0.5: # since speeder can dodge, only 50% of the time it gets hit
            Ship.shotAt(self,blast)

Define method that chooses a shooter from a bunch of ships that have not shot yet, return updated ready to shoot list

In [105]:
def choose_shooter(ready_to_shoot):
    # choose a random ship to be a shooter
    if len(ready_to_shoot) > 1:
        num = random.randint(0,len(ready_to_shoot)-1)
        shooter = ready_to_shoot[num]
        ready_to_shoot.pop(num)
    else:
        shooter = ready_to_shoot.pop()
    return shooter, ready_to_shoot

Define a method that get a shooter to randomly shoot a target and return the pair

In [106]:
import copy
def shooting_event(shooter, surviving_list):
    # create a list of possible targets, which is every surviver but shooter
    surviving_list.remove(shooter)
    possible_targets = copy.deepcopy(surviving_list) # only copy content of list
    # choose a random ship to be target
    if len(possible_targets) >1:
        num = random.randint(0,len(possible_targets)-1)
        target = possible_targets[num]
    else:
        target = possible_targets[0]
    shooter.shoot(target)
    return target

Define a method that takes the shooter and target survivor list, then generate a message, as well as an updated survivor list

In [107]:
def shooting_result(shooter, target, surviving_list):
    if target.isAlive():
        message = "{} attacked {}.".format(shooter.getName(), target.getName())
    else:
        message = "{} killed {}.".format(shooter.getName(), target.getName())
        surviving_list.remove(target)
    return message, surviving_list

Define a method that performs all events at a time step, including leting every ship shoot once, print out message of each attack, and return a updated surviving list.

In [108]:
def event_at_time_step(surviving_list):
    # create a list of ship that has not taken turn yet
    ready_to_shoot = copy.deepcopy(surviving_list) # copy content of list
    while len(ready_to_shoot) > 0: # while there is at least one person still waiting to shoot
        # randomly choose a shooter and get an updated list of ready to shoot
        shooter, ready_to_shoot = choose_shooter(ready_to_shoot)
        # play out shooting event, provide a list of surviving ships for potential tarets, return the target
        target = shooting_event(shooter,surviving_list)
        message, surviving_list = shooting_result(shooter, target, surviving_list)
        print(message)
    return surviving_list

At the end of each time step, summarize the status of each ship

In [109]:
def summary_after_time_step(surviving_list, time):
    print()
    print("At time = {}:".format(time))
    for ship in surviving_list:
        print(ship)

Main method

In [110]:
# create ships
Sam = Speeder('Sam',100,50,10) 
Jeff = WarShip('Jeff',100,50,10)
Philip = Ship('Philip',100,50,10)
Brenda = Ship('Brenda',100,50,10)
Leting = Ship('Leting',100,50,10)
# create list of surviving ships
surviving_list = [Sam, Jeff, Philip, Brenda, Leting]
# intialize time
time = 0
# at each unit time interval, each ship takes turn shooting at other ship. The order is random
# this continues as long as there is more than one surviving ship
while len(surviving_list)>1:
    surviving_list = event_at_time_step(surviving_list)
    summary_after_time_step(surviving_list, time)
    time =  time + 1

# exit loop only if one ship is left
print()
print("The victor is {}".format(surviving_list[0].getName()))

Name = Sam, Type = standard ship, Hull = 50, Shield = 100, Laser = 10
Name = Jeff, Type = war ship, Hull = 50, Shield = 100, Laser = 10
Name = Philip, Type = standard ship, Hull = 50, Shield = 100, Laser = 10
Name = Brenda, Type = standard ship, Hull = 50, Shield = 100, Laser = 10
Name = Leting, Type = standard ship, Hull = 50, Shield = 100, Laser = 10
Philip attacked Jeff.
Name = Sam, Type = standard ship, Hull = 50, Shield = 100, Laser = 10
Name = Jeff, Type = war ship, Hull = 50, Shield = 90, Laser = 10
Name = Brenda, Type = standard ship, Hull = 50, Shield = 100, Laser = 10
Name = Leting, Type = standard ship, Hull = 50, Shield = 100, Laser = 10
Sam attacked Brenda.
Name = Jeff, Type = war ship, Hull = 50, Shield = 90, Laser = 10
Name = Brenda, Type = standard ship, Hull = 50, Shield = 90, Laser = 10
Name = Leting, Type = standard ship, Hull = 50, Shield = 100, Laser = 10
Jeff attacked Leting.
Name = Brenda, Type = standard ship, Hull = 50, Shield = 90, Laser = 10
Name = Leting, Ty

IndexError: list index out of range